In [53]:
# First we import the libraries
import numpy as np
import matplotlib.pyplot as plt

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [55]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta ----------------------------#
delta = "1.04" 
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
#---------------------------------------------------------------#

train_path = "data/nY=8/δ="+delta+"_samples.csv"
train_data = data.load_data(train_path)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

torch.Size([10000, 8])

In [56]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the 
# number of visible nodes (nv) and number of higgen nodes (nh). 
# As we have an array of 8 atoms, we have 6 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the 
# data we are using and which can be varied to get optimal result. 
# For this problem, nh was varied from 4 to 8 and nh = 8 gave quite good results.
# Hence, we set nh = 8.
nh = 8

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [57]:
# Below we have more hyperparameters of the RBM model. Like the number of 
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions. 
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100
nbs = pbs
lr = 0.0065 
k = 10

# Now we train our RBM using the above parameters. 
# Note that the training process will take 2 - 3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

Total time elapsed during training: 176.272 s


In [58]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt in the output directory.
nn_state.save("output/rydberg_data.pt")
torch.load("output/rydberg_data.pt")
# Below we have the parameters  (weights and biases) of the trained RBM:

{'rbm_am': OrderedDict([('weights',
               tensor([[-0.2691, -0.1283,  0.3138, -0.4687,  0.3145, -0.4566,  0.0781, -0.0145],
                       [-0.2504,  0.1024, -0.3418,  0.3027, -0.3777, -0.1798,  0.1818, -0.0577],
                       [-0.2438,  0.3075, -0.1975, -1.0659,  1.5449, -1.6659,  0.7075, -0.3233],
                       [-0.6828,  0.2191, -0.2113,  0.0933, -0.4902,  0.5794, -0.8400,  0.7274],
                       [-1.3186,  0.3422,  0.1518, -0.3433,  0.0974,  0.0467, -0.6545,  1.0999],
                       [-0.4649,  0.5751, -1.3093,  1.4274, -1.4459,  0.6874, -0.4509, -0.1412],
                       [-0.0166, -0.0761, -0.1564,  0.2834, -0.6170,  0.3350,  0.0094, -0.2923],
                       [-0.0611, -0.4306,  0.7124, -1.4103,  1.5701, -1.6413,  0.6318, -0.1733],
                       [ 1.3345, -0.8171, -0.7828,  1.3270, -1.1631,  0.1390,  0.1938, -0.9894],
                       [-1.0072,  0.6329,  0.2257, -0.6394, -0.0790,  0.5386, -0.5446,  0.2

In [59]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM. 
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 5000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('output/reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)

In [60]:
# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hencem there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in 
# our produced sample in in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like: 
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>  
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the 
# amplitudes in amplitudeList and then also 
# prints out the amplitudes in 'output/reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (2**nv):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('output/reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

In [61]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and 
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1) 
# in each state.

# Now we get average occupation of each site by finding the 
# number of atoms in the up state in each site and then
# by dividing by the number of the produced samples 
occupation = [0]*nv
for i in range(len(sampleList)):
    j = 0
    for j in range(nv):
        occupation[j] = sampleList[i][j] + occupation[j]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

[0.3904, 0.3604, 0.4166, 0.3382, 0.42, 0.3512, 0.3904, 0.3834]

In [62]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((nv,nv))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the 
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected 
# to be nearly equal (which they are)

# Note that the matrix in data/nY=8/δ=delta_2_pt_fn.csv may be a bit misleading. 
# The matrix generated there is symmetric and this is why the values 
# below the diagonal were not computed. However the actual matrix is symmetric
TwoPointFunc

array([[ 0.23798784, -0.11810016,  0.10655936, -0.08423328,  0.079432  ,
        -0.09010848,  0.10358784, -0.12767936],
       [-0.11810016,  0.23051184, -0.11974264,  0.10051272, -0.085368  ,
         0.07282752, -0.08190016,  0.10942264],
       [ 0.10655936, -0.11974264,  0.24304444, -0.11829412,  0.106628  ,
        -0.08510992,  0.08115936, -0.09212444],
       [-0.08423328,  0.10051272, -0.11829412,  0.22382076, -0.118044  ,
         0.10742416, -0.08543328,  0.08453412],
       [ 0.079432  , -0.085368  ,  0.106628  , -0.118044  ,  0.2436    ,
        -0.121104  ,  0.114432  , -0.092428  ],
       [-0.09010848,  0.07282752, -0.08510992,  0.10742416, -0.121104  ,
         0.22785856, -0.11390848,  0.11254992],
       [ 0.10358784, -0.08190016,  0.08115936, -0.08543328,  0.114432  ,
        -0.11390848,  0.23798784, -0.12367936],
       [-0.12767936,  0.10942264, -0.09212444,  0.08453412, -0.092428  ,
         0.11254992, -0.12367936,  0.23640444]])